In [1]:
from __future__ import print_function, division
import numpy as np
import os
import gc
import sys
import datetime
import h5py
import contextlib
import math
import pandas as pd
from IPython.display import display

In [2]:
from model_specification import *
from data_interface import *
import learning_parameters

Using TensorFlow backend.


In [14]:
# Settings
evaluation_file_path = "../data/dataset/Eval_20180329_1013.h5"
dataset_partition = "testing"
interpretation_file_path = "../data/dataset/interpretation_20180204.h5"

first_stage_verbose = True

In [15]:
evaluation_data_interface = EvaluationDataInterface()
evaluation_data_interface.open_file(evaluation_file_path)
interpretation_data_interface = EvaluationDataInterface()
interpretation_data_interface.open_file(interpretation_file_path)

In [16]:
table_text, count_cross_table, prob_label_cross_table, prob_pred_cross_table = evaluation_data_interface.read_evaluation_results(dataset_partition)

label_descriptions = evaluation_data_interface.read_label_descriptions()
label_description_list = [''] * count_cross_table.shape[1]
for label_id, label_description in label_descriptions:
    label_description_list[int(label_id)] = label_description

sum_table_label = count_cross_table.sum(axis=2)
sum_table_pred = count_cross_table.sum(axis=1)



interpretation_headers, interpretation_scores = interpretation_data_interface.read_interpretation_results()


In [17]:
list(evaluation_data_interface.data_file.keys())

['devcount_cross_table',
 'devprob_label_cross_table',
 'devprob_pred_cross_table',
 'devtable_text',
 'label_description_set',
 'testingcount_cross_table',
 'testingprob_label_cross_table',
 'testingprob_pred_cross_table',
 'testingtable_text',
 'trainingcount_cross_table',
 'trainingprob_label_cross_table',
 'trainingprob_pred_cross_table',
 'trainingtable_text']

In [18]:
evaluation_data_interface.close_file()
interpretation_data_interface.close_file()

In [19]:
for i, data_name in enumerate(table_text):
    print(i, ":", data_name)

0 : overall
1 : 8
2 : 20
3 : 49
4 : 67
5 : 77
6 : 85
7 : 91
8 : 117
9 : 119


In [20]:
read_index = range(len(table_text))
# read_index = [1]

In [21]:
bacteria_type_mapping = [[1], [2], [3]]
interpretation_str_list = ["Normal", "Intermediate", "Infection"]

def get_bacteria_count_sums(bacteria_count_full, bacteria_type_mapping):
#     print(bacteria_count_full)
    results = []
    for list_labels in bacteria_type_mapping:
        count_total = 0
        for label in list_labels:
            count_total += bacteria_count_full[label]
        results.append(count_total)
    return results

def get_density_class(count):
    if count == 0:
        return 0
    elif count < 1:
        return 1
    elif count < 5:
        return 2
    elif count <= 30:
        return 3
    else:
        return 4

def get_nugent_score(lactobacillus_count, gardnerella_bacteroides_count, curved_rod_count):
    total_score = 0
    lactobacillus_score = 4 - get_density_class(lactobacillus_count)
    gardnerella_score = get_density_class(gardnerella_bacteroides_count)
    curved_rod_score = int( (get_density_class(curved_rod_count) + 1) / 2)
    return lactobacillus_score, gardnerella_score, curved_rod_score, lactobacillus_score + gardnerella_score + curved_rod_score
    
def get_nugent_score_interpretation_int(nugent_score):
    if nugent_score <= 3:
        return 0
    elif nugent_score <= 6:
        return 1
    else:
        return 2

def get_nugent_score_interpretation_str(nugent_score):
    if nugent_score <= 3:
        return "Normal"
    elif nugent_score <= 6:
        return "Intermediate"
    else:
        return "Infection"
    
@contextlib.contextmanager
def print_options_context(*args, **kwargs):
    original_print_options = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    try:
        yield
    finally: 
        np.set_printoptions(**original_print_options)
        

def dataframe_highlight_diagonal(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    num_rows = len(data)
    is_diagonal = [[x == y for x in range(num_rows)] for y in range(num_rows)]
    
    return pd.DataFrame(np.where(is_diagonal, attr, ''), index=data.index, columns=data.columns)
        
def dataframe_suppress_zero(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    attr = 'color: {}'.format('lightgray' if math.isclose(0.0, val, rel_tol=1e-5) else 'black')
    return attr
    
def get_pandas_dataframe(cross_table, axis_text_list = None, highlight_diagonal = False, fixed_width = None):
#     df = pd.DataFrame(cross_table)
    df = pd.DataFrame(cross_table, index = axis_text_list, columns = axis_text_list)
    df = df.style.applymap(dataframe_suppress_zero)
    
    if highlight_diagonal:
#         pass
        df = df.apply(dataframe_highlight_diagonal, color='yellow', axis=None)

    if not fixed_width is None:
        df = df.applymap(lambda x: str(x).ljust(fixed_width))
        df.columns = df.columns.map(lambda x: str(x).ljust(fixed_width))
        
    return df


In [22]:
lactobacillus_score_corss_table = np.zeros((5, 5), dtype=int)
gardnerella_score_cross_table = np.zeros((5, 5), dtype=int)
curved_rod_score_cross_table = np.zeros((3, 3), dtype=int)
nugent_score_cross_table = np.zeros((11, 11), dtype=int)
infection_degree_cross_table = np.zeros((3, 3), dtype=int)

with print_options_context(formatter={'float': '{:>7.1%}'.format}):
    for i in read_index:
        
        print("Result for " + table_text[i])
        if first_stage_verbose:
            print("Count cross table")
            display(get_pandas_dataframe(count_cross_table[i], label_description_list, highlight_diagonal = True, fixed_width = 11))
            print("Probability of given label")
            display(get_pandas_dataframe(prob_label_cross_table[i], label_description_list, highlight_diagonal = True)) # .style.format("{:>7.1%}")
            print("Probability of given prediction")
            display(get_pandas_dataframe(prob_pred_cross_table[i], label_description_list, highlight_diagonal = True)) # .style.format("{:>7.1%}")

        print("Prediction Nugent Score and degree of infection")
        bacteria_counts = get_bacteria_count_sums(sum_table_pred[i], bacteria_type_mapping)
        pred_lscore, pred_gscore, pred_cscore, pred_total = get_nugent_score(bacteria_counts[0], bacteria_counts[1], bacteria_counts[2])
#         print(pred_lscore, pred_gscore, pred_cscore, pred_total, bacteria_counts)
        score_interpretation_str = get_nugent_score_interpretation_str(pred_total)
        print("Score: {:d} + {:d} + {:d} = {:2d}, Interpretation: {:s}".format(pred_lscore, pred_gscore, pred_cscore, pred_total, score_interpretation_str))

        print("Label Nugent Score and degree of infection")
        bacteria_counts = get_bacteria_count_sums(sum_table_label[i], bacteria_type_mapping)
        label_lscore, label_gscore, label_cscore, label_total = get_nugent_score(bacteria_counts[0], bacteria_counts[1], bacteria_counts[2])
#         print(bacteria_counts)
        score_interpretation_str = get_nugent_score_interpretation_str(label_total)
        print("Score: {:d} + {:d} + {:d} = {:2d}, Interpretation: {:s}".format(label_lscore, label_gscore, label_cscore, label_total, score_interpretation_str))

        if table_text[i] in interpretation_scores:
            print("Source Nugent Score and degree of infection")
            source_lscore, source_gscore, source_cscore = interpretation_scores[table_text[i]]
            source_total = source_lscore + source_gscore + source_cscore
            score_interpretation_str = get_nugent_score_interpretation_str(source_total)
            print("Score: {:d} + {:d} + {:d} = {:2d}, Interpretation: {:s}".format(source_lscore, source_gscore, source_cscore, source_total, score_interpretation_str))
            
            lactobacillus_score_corss_table[source_lscore, pred_lscore] += 1
            gardnerella_score_cross_table[source_gscore, pred_gscore] += 1
            curved_rod_score_cross_table[source_cscore, pred_cscore] += 1
            nugent_score_cross_table[source_total, pred_total] += 1
            infection_degree_cross_table[get_nugent_score_interpretation_int(source_total), get_nugent_score_interpretation_int(pred_total)] += 1
        else:
            print("No source data available for " + table_text[i])
            
        print("----------------------------------------")

Result for overall
Count cross table


,Null,Lactobacillus,Gardnerella,Curved Rods,Noise
Null,0,122,451,92,216
Lactobacillus,0,26,12,0,1
Gardnerella,0,16,75,8,18
Curved Rods,0,1,3,0,0
Noise,0,0,4,1,66


Probability of given label


,Null,Lactobacillus,Gardnerella,Curved Rods,Noise
Null,0,0.138479,0.511918,0.104427,0.245176
Lactobacillus,0,0.666667,0.307692,0,0.025641
Gardnerella,0,0.136752,0.641026,0.0683761,0.153846
Curved Rods,0,0.25,0.75,0,0
Noise,0,0,0.056338,0.0140845,0.929577


Probability of given prediction


...

Probability of given label


,Null,Lactobacillus,Gardnerella,Curved Rods,Noise
Null,0,0.0182927,0.359756,0.219512,0.402439
Lactobacillus,0,0,0,0,0
Gardnerella,0,0,0.56,0.24,0.2
Curved Rods,0,0,0,0,0
Noise,0,0,0,0.0526316,0.947368


Probability of given prediction


,Null,Lactobacillus,Gardnerella,Curved Rods,Noise
Null,0,1,0.808219,0.837209,0.741573
Lactobacillus,0,0,0,0,0
Gardnerella,0,0,0.191781,0.139535,0.0561798
Curved Rods,0,0,0,0,0
Noise,0,0,0,0.0232558,0.202247


Prediction Nugent Score and degree of infection
Score: 2 + 4 + 2 =  8, Interpretation: Infection
Label Nugent Score and degree of infection
Score: 4 + 3 + 0 =  7, Interpretation: Infection
Source Nugent Score and degree of infection
Score: 2 + 4 + 1 =  7, Interpretation: Infection
----------------------------------------


In [23]:
print("Lactobacillus score cross table")
display(get_pandas_dataframe(lactobacillus_score_corss_table, highlight_diagonal = True))
print("Gardnerella score cross table")
display(get_pandas_dataframe(gardnerella_score_cross_table, highlight_diagonal = True))
print("Curved rod score cross table")
display(get_pandas_dataframe(curved_rod_score_cross_table, highlight_diagonal = True))
print("Total nugent score cross table")
display(get_pandas_dataframe(nugent_score_cross_table, highlight_diagonal = True))
print("Infection degree cross table")
display(get_pandas_dataframe(infection_degree_cross_table, interpretation_str_list, highlight_diagonal = True))

Lactobacillus score cross table


,0,1,2,3,4
0,0,0,0,0,0
1,0,2,0,0,0
2,0,0,1,0,0
3,1,1,1,0,0
4,1,1,0,0,1


Gardnerella score cross table


,0,1,2,3,4
0,0,0,1,0,0
1,0,0,0,0,0
2,0,0,0,1,0
3,0,0,0,2,1
4,0,0,0,1,3


Curved rod score cross table


,0,1,2
0,2,1,1
1,1,2,1
2,0,0,1


Total nugent score cross table


,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,2,0,0,0,0,0
6,0,0,0,1,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,0,1,0


Infection degree cross table


,Normal,Intermediate,Infection
Normal,1,0,0
Intermediate,1,2,1
Infection,0,2,2


In [35]:
temp_nugent_score_cross_table = nugent_score_cross_table
temp_infection_degree_cross_table = infection_degree_cross_table

In [55]:
temp_nugent_score_cross_table += nugent_score_cross_table
temp_infection_degree_cross_table += infection_degree_cross_table

In [56]:
print("Total nugent score cross table")
display(get_pandas_dataframe(temp_nugent_score_cross_table, highlight_diagonal = True))
print("Infection degree cross table")
display(get_pandas_dataframe(temp_infection_degree_cross_table, interpretation_str_list, highlight_diagonal = True))

Total nugent score cross table


,0,1,2,3,4,5,6,7,8,9,10
0,0,0,1,1,0,0,0,0,0,0,0
1,0,3,1,9,5,3,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,2,2,3,4,2,1,0,0
5,0,0,1,0,2,4,2,1,0,0,0
6,0,0,0,1,0,1,7,3,1,0,0
7,0,0,0,0,0,0,5,7,3,2,0
8,0,0,0,0,0,0,1,8,4,3,0
9,0,0,0,0,0,2,3,7,7,1,2


Infection degree cross table


,Normal,Intermediate,Infection
Normal,15,9,0
Intermediate,5,25,8
Infection,0,13,44


# Specific Samples

In [4]:
from keras.models import Model, load_model
from matplotlib import pyplot as plt
import cv2

In [5]:
def cv_img_size(img):
    return tuple(img.shape[1::-1])

def plotCVColorImage(image, size = (800, 800)):
    converted_size = (size[0] / 100, size[1] / 100)
    plt.figure(figsize=converted_size, dpi = 100)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()

def plotRGBColorImage(image, size = (800, 800)):
    converted_size = (size[0] / 100, size[1] / 100)
    plt.figure(figsize=converted_size, dpi = 100)
    plt.axis("off")
    plt.imshow(image)
    plt.show()

In [6]:
model_file_name = "saved_models/Wide0325.00300.h5"

In [7]:
model_parameters = learning_parameters.get_model_parameters()
file_parameters = learning_parameters.get_file_parameters()
training_parameters = learning_parameters.get_training_parameters()
model = load_model(model_file_name)

Learning rate:  0.001


In [8]:
dim_x = model_parameters['input_shape'][0]
dim_y = model_parameters['input_shape'][1]
dim_z = model_parameters['input_shape'][2]

file_used = file_parameters['full_data_file_path']
batch_size = training_parameters['batch_size']
datagen = ModelDataInterface(file_used, dim_x = model_parameters['input_shape'][0], 
    dim_y = model_parameters['input_shape'][1], dim_z = model_parameters['input_shape'][2], 
    n_classes = model_parameters['num_output_classes'])

# stage_texts = ['training', 'dev', 'testing']
stage_text = 'training'
n_classes = model_parameters['num_output_classes']

In [ ]:
data_gen = datagen.get_sequence_generator(stage_text, data_only = True, label_only = False, 
                    data_augmentation_enabled = False, shuffle = False, batch_size = batch_size, cached = True)

predictions = model.predict_generator(generator = data_gen,
                                workers = 1,
                                verbose = 0,
                                use_multiprocessing = False)
predictions_flattened = np.argmax(predictions, axis = 1)


Preparing sequence generator for 'training'


In [9]:
label_gen = datagen.get_sequence_generator(stage_text, data_only = False, label_only = True, 
                                    data_augmentation_enabled = False, shuffle = False, batch_size = batch_size, cached = False)
labels_flattened = np.array([], dtype = 'uint8')
for i in range(len(label_gen)):
    labels_flattened = np.append(labels_flattened, label_gen.get_batch(i))


Preparing sequence generator for 'training'


In [78]:
data_flattened = np.empty([0,dim_x,dim_y,dim_z])
for i in range(len(data_gen)): # len(data_gen)
    data_flattened = np.append(data_flattened, data_gen.get_batch(i), axis = 0)

(0, 224, 224, 3)


In [13]:
file_meta_data = datagen.get_file_metadata(stage_text)
count_offset = 0
for image_name, label_count in file_meta_data:
    if image_name == '3':
        for i in range(label_count):
#             plotRGBColorImage(data_flattened[i + count_offset], (200, 200))
#             print("P:" + str(predictions_flattened[i + count_offset]) + " L:" + str(labels_flattened[i + count_offset]))
            print(" L:" + str(labels_flattened[i + count_offset]))
    count_offset += label_count

 L:4
 L:0
 L:4
 L:4
 L:0
 L:1
 L:0
 L:0
 L:0
 L:0
 L:1
 L:1
 L:0
 L:1
 L:0
 L:1
 L:0
 L:0
 L:0
 L:1
 L:0
 L:0
 L:0
 L:1
 L:1
 L:0
 L:0
 L:0
 L:0
 L:0
 L:1
 L:0
 L:0
 L:0
 L:0
 L:0
 L:1
 L:0
 L:0
 L:0
 L:1
 L:0
 L:0
 L:1
 L:1
 L:1
 L:0
 L:0
 L:1
 L:1
 L:1
 L:1
 L:1
 L:1
 L:1
 L:1
 L:0
 L:1
 L:1
 L:1
 L:0
 L:1
 L:1
 L:0
 L:1
 L:1
 L:0
 L:0
 L:0
 L:0
 L:0
 L:1
 L:0
 L:0
 L:1
 L:1
 L:1
 L:1
 L:1
 L:1
 L:0
 L:1
 L:1
 L:1
 L:1
 L:1
 L:1
 L:1
 L:1
